In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model import unet, segnet_vgg16, fcn_vgg16_8s, VGGUnet2, D_resunet, D_resunet1
from data import trainGenerator, testGenerator, saveResult, testGenerator2
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard
import keras.backend as K
import os, cv2
import numpy as np
import skimage.io as io
import skimage.transform as trans

In [11]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 23 17:07:33 2019
Note! crop the images and masks need to change several lines code(60/63)
@author: zetn
"""
from model import unet, segnet_vgg16, fcn_vgg16_8s, VGGUnet2, res_unet, D_resunet1
from data import trainGenerator, testGenerator, saveResult, testGenerator2
from keras.callbacks import ModelCheckpoint
import keras.backend as K
import os, cv2
import numpy as np
import skimage.io as io
import skimage.transform as trans

#fileDir = "data/membrane/test/sub_test/mask8"  #test images(1024*1024)
fileDir = "data/membrane/IEEE_road/train/sat_img/"
preDir = "data/membrane/IEEE_road/train/sat_img_crops/" #Dir of predict mask

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
def crop_image(src, save_path):
    TEST_SET = os.listdir(src)
    img_h = 256
    img_w = 256
    stride = img_h-40
    for n in range(len(TEST_SET)):
        image_name = TEST_SET[n]
        path1 = image_name[0:-7]+'mask.png'  #rename mask
        # load the image
        # image = cv2.imread(os.path.join(src,image_name), cv2.IMREAD_UNCHANGED)
        # image = cv2.imread(os.path.join(src,image_name))
        print(os.path.join(src,image_name))
        image = io.imread(os.path.join(src,image_name))
        
        print(image.shape)
        h, w, _ = image.shape
        # h, w = image.shape

        num = 0;
        #image = img_to_array(image)
        # padding_img = (padding_img - np.min(padding_img)) / (np.max(padding_img) - np.min(padding_img))

        print('[{}/{}], croping:{}'.format(n+1, len(TEST_SET), image_name))

        #mask_whole = np.zeros((h, w, 1), dtype=np.uint8)
        #temp = np.zeros((img_h, img_h), dtype=np.uint8)

        for i in range(0, (h // stride)+1):
            for j in range(0, (w // stride)+1):
                h_begin = i * stride
                w_begin = j * stride
                
                if h_begin + img_h > h:
                    h_begin = h_begin - (h_begin + img_h - h)
                
                if w_begin + img_w > w:
                    w_begin = w_begin - (w_begin + img_w - w)
                
                crop = image[h_begin:h_begin + img_h, w_begin:w_begin + img_w] 
                if num <= 9:
                    #path1 = image_name[0:-4]+'0'+ str(num)+'.jpg'
                    path1 = image_name[0:-4]+'0'+ str(num)+'.png'
                else:
                    #path1 = image_name[0:-4]+str(num)+'.jpg'
                    path1 = image_name[0:-4]+str(num)+'.png'
                
                #io.imsave(save_path + path1, crop)
                cv2.imwrite(save_path + path1, crop)
                num = num + 1
        print('Done!')
crop_image(fileDir, preDir)               


data/membrane/IEEE_road/train/sat_img/104_sat.jpg
(1024, 1024, 3)
[1/50], croping:104_sat.jpg
Done!
data/membrane/IEEE_road/train/sat_img/1054_sat.jpg
(1024, 1024, 3)
[2/50], croping:1054_sat.jpg
Done!
data/membrane/IEEE_road/train/sat_img/113_sat.jpg
(1024, 1024, 3)
[3/50], croping:113_sat.jpg
Done!
data/membrane/IEEE_road/train/sat_img/1849_sat.jpg
(1024, 1024, 3)
[4/50], croping:1849_sat.jpg
Done!
data/membrane/IEEE_road/train/sat_img/1934_sat.jpg
(1024, 1024, 3)
[5/50], croping:1934_sat.jpg
Done!
data/membrane/IEEE_road/train/sat_img/1945_sat.jpg
(1024, 1024, 3)
[6/50], croping:1945_sat.jpg
Done!
data/membrane/IEEE_road/train/sat_img/2129_sat.jpg
(1024, 1024, 3)
[7/50], croping:2129_sat.jpg
Done!
data/membrane/IEEE_road/train/sat_img/2262_sat.jpg
(1024, 1024, 3)
[8/50], croping:2262_sat.jpg
Done!
data/membrane/IEEE_road/train/sat_img/2495_sat.jpg
(1024, 1024, 3)
[9/50], croping:2495_sat.jpg
Done!
data/membrane/IEEE_road/train/sat_img/2647_sat.jpg
(1024, 1024, 3)
[10/50], croping:26

In [14]:
data_gen_args = dict(rotation_range=90.,
                    #width_shift_range=0.1,
                    #height_shift_range=0.1,
                    #shear_range=0.1,
                    #zoom_range=0.1,
                    fill_mode='nearest',
                    horizontal_flip=True,
                    vertical_flip=True)

train_Gene = trainGenerator(2,'data/membrane/IEEE_road/train/','sat_img_crops','masks_crops',data_gen_args,save_to_dir = None)
val_Gene = trainGenerator(2,'data/membrane/IEEE_road/test/','sat_img_crops','masks_crops',data_gen_args)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor=0.2, patience=3, verbose=0, mode='min', epsilon=1e-4, 
                              cooldown=0, min_lr=1e-6)
visual = TensorBoard(log_dir='./D_resunet1_log', histogram_freq=0, write_graph=True, write_images=True)
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='min')
#model = unet()
#model = segnet_vgg16()
#model = fcn_vgg16_8s()
#model.load_weights('fcn_vgg16_8s.hdf5')
#model = fcn_vgg16_8s()
#model = VGGUnet2()
model = D_resunet()

#model = res_unet1()
#model.load_weights('res_unet.hdf5')

model_checkpoint = ModelCheckpoint('D_Resunet.hdf5', monitor='val_loss',verbose=1, save_best_only=True)
model.fit_generator(train_Gene,steps_per_epoch=50,epochs=10,
                    callbacks=[model_checkpoint, visual, reduce_lr, earlystop], 
                    validation_data=val_Gene, validation_steps=220)#step_per_epoch and validation_steps equals to number of samples divide batchsize


Found 1250 images belonging to 1 classes.
Found 1250 images belonging to 1 classes.


C:\Users\subha\miniconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\subha\AppData\Local\Temp\ipykernel_46236\2489772996.py:29: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_Gene,steps_per_epoch=50,epochs=10,


Epoch 1/10
50/50 [==============================] - ETA: 0s - loss: 1.1772 - IoU: 0.0955Found 375 images belonging to 1 classes.
Found 375 images belonging to 1 classes.

Epoch 1: val_loss improved from inf to 1.33775, saving model to D_Resunet.hdf5
50/50 [==============================] - 82s 2s/step - loss: 1.1772 - IoU: 0.0955 - val_loss: 1.3377 - val_IoU: 0.0260 - lr: 2.0000e-04
Epoch 2/10
50/50 [==============================] - ETA: 0s - loss: 1.1136 - IoU: 0.0663
Epoch 2: val_loss improved from 1.33775 to 1.14118, saving model to D_Resunet.hdf5
50/50 [==============================] - 100s 2s/step - loss: 1.1136 - IoU: 0.0663 - val_loss: 1.1412 - val_IoU: 0.0638 - lr: 2.0000e-04
Epoch 3/10
50/50 [==============================] - ETA: 0s - loss: 1.1025 - IoU: 0.0926
Epoch 3: val_loss improved from 1.14118 to 1.13564, saving model to D_Resunet.hdf5
50/50 [==============================] - 98s 2s/step - loss: 1.1025 - IoU: 0.0926 - val_loss: 1.1356 - val_IoU: 0.0277 - lr: 2.0000e-

In [15]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 11 12:12:34 2019
带重叠的滑动窗口patch（256*256）预测并缝合形成大的mask（1024*1024）图片
@author: zetn
"""

from model import unet, segnet_vgg16, fcn_vgg16_8s, VGGUnet2, res_unet, D_resunet1
from data import trainGenerator, testGenerator, saveResult, testGenerator2
from keras.callbacks import ModelCheckpoint
import keras.backend as K
import os, cv2
import numpy as np
import skimage.io as io
import skimage.transform as trans

fileDir = "data/membrane/IEEE_road/test/test_images/"  #test images(1024*1024)
#fileDir = "data/membrane/train/f"
preDir = "data/membrane/IEEE_road/test/sub_test/predict/" #Dir of predict mask



#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
def predict_z(src, predict_path):
    TEST_SET = os.listdir(src)
    model = D_resunet()
    #model = res_unet1()
    print('Loading Model weights...')
    model.load_weights('D_resunet.hdf5')
    print('completed!')
    img_h = 256
    img_w = 256
    stride = img_h-16
    for n in range(len(TEST_SET)):
        path = TEST_SET[n]
        path1 = path[0:-7]+'mask.png'  #rename mask
        # load the image
        image = io.imread(os.path.join(src,path))
        h, w, _ = image.shape

        image = image / 255.0
        #image = img_to_array(image)
        # padding_img = (padding_img - np.min(padding_img)) / (np.max(padding_img) - np.min(padding_img))

        print('[{}/{}], predicting:{}'.format(n+1, len(TEST_SET), path))

        mask_whole = np.zeros((h, w, 1), dtype=np.uint8)
        #temp = np.zeros((img_h, img_h), dtype=np.uint8)

        for i in range(0, (h // stride)+1):
            for j in range(0, (w // stride)+1):
                h_begin = i * stride
                w_begin = j * stride
                
                if h_begin + img_h > h:
                    h_begin = h_begin - (h_begin + img_h - h)
                
                if w_begin + img_w > w:
                    w_begin = w_begin - (w_begin + img_w - w)
                
                crop = image[h_begin:h_begin + img_h, w_begin:w_begin + img_w, :3] #[****)
                
                ch, cw, _ = crop.shape

                if ch != img_h or cw != img_h:
                    print('invalid size!')
                    print(i, j, h_begin, w_begin, ch, cw)
                    break
                
                crop = np.expand_dims(crop, axis=0)
                pred = model.predict(crop, verbose=2)
                pred = pred.reshape((img_h, img_h, 1)).astype(np.float64)
                #pred = np.argmax(pred, axis=2)
                #print(pred.shape)
                #pred = np.array(pred)
                
                pred[pred >= 0.5] = 1
                pred[pred < 0.5] = 0
                pred = pred * 255
                '''
                for a in range(img_h):
                    for b in range(img_h):
                        if pred[a, b] == 0.:
                            temp[a, b, :] = [223, 223, 223]
                        elif pred[a, b] == 1.:
                            temp[a, b, :] = [255, 204, 163]
                        else:
                            print('Unknown type:', pred[a, b])
                '''
                mask_whole[h_begin:h_begin + img_h, w_begin:w_begin + img_w] \
                    = pred
                # + mask_whole[i * stride:i * stride + image_size, j * stride:j * stride + image_size, :]
        cv2.imwrite(predict_path + path1, mask_whole[0:h, 0:w])
        #print('Done!')
        
predict_z(fileDir, preDir)        


C:\Users\subha\miniconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Loading Model weights...
completed!
[1/4], predicting:355_sat.jpg
1/1 - 0s - 399ms/epoch - 399ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 21ms/epoch - 21ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 21ms/epoch - 21ms/step
1/1 - 0s - 21ms/epoch - 21ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 23ms/epoch - 23ms/step
1/1 - 0s - 20ms/epoch - 20ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 20ms/epoch - 20ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 21ms/epoch - 21ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 21ms/epoch - 21ms/step
[2/4], predicting:48631_sat.jpg
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 18ms/